In [ ]:
# Test inverse.jl using the file inverse.inverse_chease4_run
using Pkg
Pkg.activate("..") 
Pkg.instantiate()
using JPEC, Plots
gr() 

In [ ]:
# 1. Define the input parameters for the equilibrium solver.
#    - eq_filename: The name of the g-file we just created.
#    - eq_type: "efit" for a standard g-file.
#    - jac_type: "boozer" or "hamada" for the output coordinates.
#    - mpsi, mtheta: Resolution of the output grid.
equil_input = JPEC.Equilibrium.EquilibriumConfig(control=JPEC.Equilibrium.EquilibriumControl(;eq_filename="inverse.chease.10",
    eq_type="chease2", jac_type="hamada", mpsi=128, mtheta=256),
    output=JPEC.Equilibrium.EquilibriumOutput())

# 2. Run the main equilibrium setup function.
#    This will read the file, solve the direct problem, and return the final object.
println("Starting equilibrium reconstruction...")
plasma_eq = JPEC.Equilibrium.setup_equilibrium(equil_input)
println("Equilibrium reconstruction complete.")


In [ ]:
# The plasma_eq object contains the final 1D profile spline, `sq`.
# We will evaluate it on a fine grid to get smooth plot lines.
psi_norm_grid = range(0.0, 1.0, length=200)

# spline_eval returns the function values (and derivatives if requested).
# For sq, the quantities are: 1:F, 2:P*mu0, 3:Toroidal Flux, 4:q
f_profiles = JPEC.spline_eval(plasma_eq.sq, collect(psi_norm_grid))

# Extract each profile into its own variable for clarity
F_profile      = f_profiles[:, 1]
P_profile      = f_profiles[:, 2]
tor_flux_prof  = f_profiles[:, 3]
q_profile      = f_profiles[:, 4]

In [ ]:

p = plot(layout=(2, 2), size=(900, 700), margin=5Plots.mm)

# Plot 1: F = R*B_phi
plot!(p[1], psi_norm_grid, F_profile,
    xlabel="Normalized Poloidal Flux (ψ_norm)",
    ylabel="F [T⋅m]",
    title="Toroidal Field Function (F)",
    legend=false,
    lw=2
)

# Plot 2: Pressure
plot!(p[2], psi_norm_grid, P_profile,
    xlabel="Normalized Poloidal Flux (ψ_norm)",
    ylabel="μ₀P [T²]",
    title="Pressure Profile",
    legend=false,
    lw=2
)

# Plot 3: Safety Factor (q)
plot!(p[3], psi_norm_grid, q_profile,
    xlabel="Normalized Poloidal Flux (ψ_norm)",
    ylabel="q",
    title="Safety Factor (q) Profile",
    legend=false,
    lw=2
)

# Plot 4: Toroidal Flux
plot!(p[4], psi_norm_grid, tor_flux_prof,
    xlabel="Normalized Poloidal Flux (ψ_norm)",
    ylabel="Φ / (2π)² [Wb]",
    title="Toroidal Flux Profile",
    legend=false,
    lw=2
)

# Display the final plot
p

In [ ]:
# block 6
# --- Plot Straight-Field-Line Angle (θ) Contours ---

p_theta = contour(
    R_grid, 
    Z_grid, 
    THETA_values_on_grid,
    title="Straight-Field-Line Angle (θ) Contours",
    xlabel="R [m]",
    ylabel="Z [m]",
    levels=20,
    c=:hsv,
    colorbar_title="Normalized θ",
    aspect_ratio=:equal,
    linewidth=1.5
)

# Mark the magnetic axis
scatter!(p_theta, [plasma_eq.ro], [plasma_eq.zo], label="Magnetic Axis", mc=:black, ms=5)

display(p_theta)